In [12]:
%matplotlib notebook
import pandas as pd
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt

df = pd.read_csv("Output/cleaned_venue_data2.csv")
df.count()

Neighborhood              1226
Neighborhood Latitude     1226
Neighborhood Longitude    1226
Venue Name                1226
Venue ID                  1226
Venue Category            1226
Venue Latitude            1226
Venue Longitude           1226
Venue City                1106
Venue State               1226
Likes                      950
Rating                     183
dtype: int64

In [11]:
# 1 - What are the most popular neighborhoods for music venues in Manhattan? 
# rank neighborhoods in by count of venues bar chart

grouped = df.groupby('Neighborhood')
grouped.count()

,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue ID,Venue Category,Venue Latitude,Venue Longitude,Venue City,Venue State,Likes,Rating
Neighborhood,,,,,,,,,,,
Arden Heights,1,1,1,1,1,1,1,1,1,0,0
Arrochar,1,1,1,1,1,1,1,1,1,0,0
Astoria Heights,4,4,4,4,4,4,4,4,4,0,0
Auburndale,7,7,7,7,7,7,7,6,7,7,1
Bath Beach,7,7,7,7,7,7,7,7,7,7,0
Bay Ridge,4,4,4,4,4,4,4,4,4,4,1
Baychester,3,3,3,3,3,3,3,3,3,3,0
Bayside,2,2,2,2,2,2,2,2,2,2,1
Bedford Park,11,11,11,11,11,11,11,9,11,11,0


In [ ]:
# 2 - What are the most popular venues by neighborhood? 


In [ ]:
# 3 - Which neighborhoods have the most highly rated/most liked venues? 


In [ ]:
# 4 - Which music venues have the most likes/highest ratings?